In [1]:
import os
import json

import pandas as pd
import numpy as np
from numpy.random import uniform
from TMM import *
from sklearn.svm import SVR
import scanpy as sc

In [2]:
simulation_number = 1000
random_selection_proportion = 0.1 # proportion of randomly selection from atlas

In [4]:
liver_cell_atlas = pd.read_csv('Atlas/GSE115469.csv.gz',index_col=0)
cell_type_table = pd.read_csv('Atlas/GSE115469.obs.csv',index_col=0)

In [5]:
cell_type_table.head()

,Sample,Cell#,Cluster#,CellType,CellType_Global,CellType_sub,Non-immune,Non-immune Separate,Immune Separate,Immune Merged
P1TLH_AAACCTGAGCAGCCTC_1,P1TLH,AAACCTGAGCAGCCTC,12,Central_venous_LSECs,LSEC,Central_venous_LSECs,LSEC,Central_venous_LSECs,Other,Other
P1TLH_AAACCTGTCCTCATTA_1,P1TLH,AAACCTGTCCTCATTA,17,Cholangiocytes,Cholangiocytes,Cholangiocytes,Cholangiocytes,Cholangiocytes,Other,Other
P1TLH_AAACCTGTCTAAGCCA_1,P1TLH,AAACCTGTCTAAGCCA,12,Central_venous_LSECs,LSEC,Central_venous_LSECs,LSEC,Central_venous_LSECs,Other,Other
P1TLH_AAACGGGAGTAGGCCA_1,P1TLH,AAACGGGAGTAGGCCA,10,Non-inflammatory_Macrophage,Macrophage,Non-inflammatory_Macrophage,Other,Other,Non-inflammatory_Macrophage,Macrophage
P1TLH_AAACGGGGTTCGGGCT_1,P1TLH,AAACGGGGTTCGGGCT,2,alpha-beta_T_Cells,T_Cells,alpha-beta_T_Cells,Other,Other,alpha-beta_T_Cells,alpha-beta_T_Cells


In [6]:
cell_annotation_column = 'CellType_sub'
cell_types = list(set(cell_type_table[cell_annotation_column]))

In [7]:
def cell_expr_generator(cell):
    cell = cell_type
    cell_id_list = list(cell_type_table[cell_type_table[cell_annotation_column] == cell].index.values)
    other_id_list = list(cell_type_table[cell_type_table[cell_annotation_column] != cell].index.values)
    random_selection_number = int(len(cell_id_list) * random_selection_proportion)
    if random_selection_number < 5:
        random_selection_number = 5
    else:
        pass
    random_selection = np.random.choice(cell_id_list,random_selection_number)
    selected_cell_reference = liver_cell_atlas.filter(random_selection).mean(axis=1)
    other_selection = np.random.choice(other_id_list,1000)
    other_cell_reference = liver_cell_atlas.filter(other_selection).mean(axis=1)
    scaling = np.random.uniform(0,1)
    proportion = [scaling,1-scaling]
    sim_expr = selected_cell_reference * proportion[0] * 100 + other_cell_reference * proportion[1] * 100
    return sim_expr, proportion

def simulation_expr():
    sim_expr = {}
    proportion = []
    sim_list = []
    for i in range(simulation_number):
        sim_id = 'Sim-%s'%(i + 1)
        sim_list.append(sim_id)
        pseudo_bulk = cell_expr_generator(cell_type)
        sim_expr[sim_id] = pseudo_bulk[0]
        proportion.append(pseudo_bulk[1])
    sim_expr = pd.DataFrame.from_dict(data = sim_expr)
    sim_expr = cpm(sim_expr)
    proportion = np.array(proportion)
    proportion = pd.DataFrame(data = proportion, columns = [cell_type,'Other'])
    proportion.index = sim_list
    return sim_expr,proportion

In [18]:
cell_type = 'Hepatocyte'

In [19]:
simulation = simulation_expr()
sim_expr = simulation[0]
proportion = simulation[1]
training_list = list(sim_expr.columns.values)

In [21]:
log2FC_threshold = 0
pct_gap_threshold = 1

markers = pd.read_csv('Seurat_Markers/GSE115469.CellType_sub.All_Markers.csv',index_col=0)

markers['pct.gap'] = markers['pct.1'] / markers['pct.2']
markers = markers[markers['avg_log2FC'] > log2FC_threshold]
markers = markers[markers['pct.gap'] > pct_gap_threshold]
cell_markers = markers[markers['cluster'] == cell_type]
marker_genes = list(cell_markers['gene'])
len(marker_genes)

188

In [22]:
bulk_list = os.listdir('Bulk')

for file in bulk_list:
    prediction_expr = pd.read_csv('Bulk/%s'%file,index_col=0)
    prediction_list = list(prediction_expr.columns.values)
   
    normalization = sim_expr.merge(prediction_expr,left_index=True,right_index=True,how='inner')
    normalization = normalization.fillna(0)
    normalization = cpm(normalization)
    tmm = calcNormfactors(normalization)
    normalization = normalization / tmm

    normalization = normalization.reindex(marker_genes)
    normalization = normalization.fillna(0)
    feature = []
    running_length = len(normalization)
    for i in range(running_length):
        line = normalization.iloc[i]
        scaled_line = line / max(line)
        feature.append(scaled_line)
    feature = pd.DataFrame(data=feature)
    feature = feature.fillna(0)
    training_feature = feature.filter(training_list)
    training_target = proportion
    regr = SVR(degree=10, C=1.0, epsilon=0)
    X = np.array([training_feature[sim] for sim in training_feature.columns.values])
    y = np.array(training_target[training_target.columns.values[0]])
    regr.fit(X,y)
    prediction_feature = feature.filter(prediction_list)
    prediction_X = np.array([prediction_feature[sample] for sample in prediction_feature.columns.values])
    prediction = regr.predict(prediction_X)
    prediction = pd.DataFrame(data=prediction,columns=[cell_type],index=prediction_list)
    
    prediction.to_csv('SVR_Estimation/%s.%s.SVR_Estimation.csv'%(file.split('.')[0],cell_type))


experiment_log = {}
experiment_log['log2FC_threshold'] = log2FC_threshold
experiment_log['pct_gap_threshold'] = pct_gap_threshold
experiment_log['Real Feature Number'] = len(marker_genes)
experiment_log['Marker Genes'] = marker_genes
with open("SVR_Estimation/experiment_log.json", "w") as outfile:
    json.dump(experiment_log, outfile)